<H1>Neighborhoods in Toronto Week 3 Part 2 - Stephane Faure</H1>

<p>Imports...</p>


In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

<p>Loading data</p>

In [2]:
doc_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(doc_url).text
soup = BeautifulSoup(source, 'lxml')

<p>Looking for the table and diving into its body</p>

In [3]:
#Since there is only one table and its class is "wikitable sortable", let's extract it
table = soup.find("table",class_="wikitable sortable")
table_body = soup.find("tbody")


In [4]:
#Extract all raws
table_rows = table_body.find_all("tr")
# Preparing a dictionnary with postal code as key
cp_dict=dict()

# Loop over postocde list excluding the fist line (title)
for wiki_postcode in table_rows[1:]:
    postcode_data = wiki_postcode.find_all("td")
    # Removong rows with not assigned entries
    if postcode_data[1].text.rstrip() == "Not assigned" or postcode_data[2].text.rstrip()== "Not assigned" :    
        continue
  
    postcode = postcode_data[0].text
    # Process the case where there is a href or not in borough description
    if postcode_data[1].find("a"):
        borough = postcode_data[1].find("a").text
    else:
        borough = postcode_data[1].text

    neighborough_data = postcode_data[2].text.rstrip()

    # Update dictionnary according if there is already an entry for the postal code or if it needs to be created
    if postcode in cp_dict:
        cp_dict[postcode][1]=cp_dict[postcode][1]+","+neighborough_data
    else:
        cp_dict[postcode]=[borough,neighborough_data]

<p>Create DataFrame</p>

In [5]:
#Change dictionary into a list
cplist=[]
for i in cp_dict.keys():
  cplist.append((i,cp_dict[i][0],cp_dict[i][1]))
# Load Dataframe from list
cols = ["Postcode", "Borough", "Neiborough"]
cp = pd.DataFrame.from_records(cplist, columns=cols)


<p>Display the shape of the DataFrame and save it</p>

In [6]:
print(cp.shape)
cp.to_csv("Toronto.csv")
cp.head()

(102, 3)


,Postcode,Borough,Neiborough
0,M6J,West Toronto,"Little Portugal,Trinity"
1,M5M,North York,"Bedford Park,Lawrence Manor East"
2,M6H,West Toronto,"Dovercourt Village,Dufferin"
3,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights"
4,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."


<h1>Part 2</h1>

In [7]:
# install Geocoder
!conda install -c conda-forge geocoder
import geocoder

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00   1.32 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00   7.26 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  34.44 MB/s


<p>Make coordonates lookup a function</p>

In [12]:
def getCoordsfromPostalCode(input_pc):
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(input_pc))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return latitude,longitude

<p>Add columns to original dataframe and update them</p>

In [14]:
cp["Latitude"] = np.nan
cp["Longitude"] = np.nan

for i in cp_dict.keys():
    lat,lon = getCoordsfromPostalCode(i)
  
    cp.loc[cp["Postcode"] == i, ["Latitude"]] = lat
    cp.loc[cp["Postcode"] == i, ["Longitude"]] = lon    


<P>Verify coordonates has been added to the DataFrame</p>

In [15]:
cp.head()

,Postcode,Borough,Neiborough,Latitude,Longitude
0,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750
1,M5M,North York,"Bedford Park,Lawrence Manor East",43.733283,-79.419750
2,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
3,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259
4,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724


In [17]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py_0 conda-forge
    branca:  0.3.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

branca-0.3.0-p 100% |################################| Time: 0:00:00 803.70 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00 927.09 kB/s
altair-2.2.2-p 100% |################################| Time: 0:00:00   1.16 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00   1.55 MB/s
Folium installed and imported!


In [58]:
# define the world map centered around Toronto, Canada with a higher zoom level
# Toronto https://github.com/jasonicarter/toronto-geojson/blob/master/toronto_crs84.geojson
tr_map = folium.Map(location=[43.653908,-79.384293], zoom_start=11)
cp.set_index("Postcode")
tr_feature_group = folium.map.FeatureGroup()

# Add post code locations on the map
for i in cp_dict.keys():
    # folium_location=[cp.loc[cp["Postcode"] == i, cp["Postcode"]["Latitude"]],cp.loc[cp["Postcode"] == i, ["Longitude"]]]
    lat = cp.loc[cp["Postcode"] == i, ["Latitude"]].get_values()
    lon = cp.loc[cp["Postcode"] == i, ["Longitude"]].get_values() 
    folium_location=[lat[0][0],lon[0][0]]
    # Add one marker per postal code location
    tr_feature_group.add_child(folium.Marker(location=folium_location,popup=i))

tr_feature_group.add_to(tr_map)

tr_map